In [36]:
import os
import urllib.request
import datetime
import json
from dotenv import load_dotenv

load_dotenv()

client_id= os.getenv('NAVER_CLIENT_ID')
client_secret= os.getenv('NAVER_CLIENT_SECRET')
#[CODE 1]
def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [37]:
#[CODE 2]
def getNaverSearch(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start,display)
    url = base + node + parameters
    responseDecode = getRequestUrl(url) #[CODE 1]
    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)


In [38]:
#[CODE 3]
def getPostData(post, jsonResult, cnt):
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']

    pdate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pdate = pdate.strftime('%Y-%m-%d %H:%M:%S')

    jsonResult.append({'cnt':cnt, 'title':title, 'description': description, 'org_link':org_link, 'link': link, 'pdate':pdate})
    return

def getPostData_Blog(post, jsonResult, cnt):
    title = post['title']
    description = post['description']
    org_link = post['bloggerlink']
    link = post['link']
    pDate = datetime.datetime.strptime(post['postdate'], '%Y%m%d')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')
    jsonResult.append({'cnt':cnt, 'title':title, 'description': description, 'org_link':org_link, 'link': link, 'pDate':pDate})
    return
# "title"
#"description"
# "link":"
# "bloggername":"하루집사의 IT한마당",
# "bloggerlink":"blog.naver.com\/dltmdgns2525",
# "postdate":"20250218"


In [39]:
#[CODE 0]
def main():
    node = 'news' #크롤링할 대상
    srcText = input('검색어를 입력하세요: ')
    cnt = 0
    jsonResult = []
    jsonResponse = getNaverSearch(node, srcText, 1, 100) #[CODE 2]
    total = jsonResponse['total']
    while ((jsonResponse != None) and (jsonResponse['display'] != 0)):
        for post in jsonResponse['items']:
            cnt += 1
            getPostData(post, jsonResult, cnt) #[CODE 3]
            # getPostData_Blog(post, jsonResult, cnt) #[CODE 3]
        start = jsonResponse['start'] + jsonResponse['display']
        jsonResponse = getNaverSearch(node, srcText, start, 100) #[CODE 2]

    print('전체 검색 : %d 건' %total)

    with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult, indent = 4, sort_keys = True, ensure_ascii = False)
        outfile.write(jsonFile)

    print("가져온 데이터 : %d 건" %(cnt))
    print('%s_naver_%s.json SAVED' % (srcText, node))

if __name__ == '__main__':
    main()

[2025-07-07 15:23:56.358128] Url Request Success
[2025-07-07 15:23:56.487693] Url Request Success
[2025-07-07 15:23:56.629789] Url Request Success
[2025-07-07 15:23:56.773612] Url Request Success
[2025-07-07 15:23:56.934956] Url Request Success
[2025-07-07 15:23:57.102589] Url Request Success
[2025-07-07 15:23:57.274494] Url Request Success
[2025-07-07 15:23:57.445151] Url Request Success
[2025-07-07 15:23:57.618430] Url Request Success
[2025-07-07 15:23:57.809096] Url Request Success
HTTP Error 400: Bad Request
[2025-07-07 15:23:57.875520] Error for URL : https://openapi.naver.com/v1/search/news.json?query=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5%EC%82%AC%EA%B4%80%ED%95%99%EA%B5%90&start=1001&display=100
전체 검색 : 11981 건
가져온 데이터 : 1000 건
인공지능사관학교_naver_news.json SAVED
